In [1]:
import pandas as pd
import json
from py2neo import Graph, Node, Relationship
import numpy as np
import time

In [2]:
graph = Graph("bolt://localhost:7687", auth=('neo4j', 'password'))

# Run STREAMING PageRank (1) on each year from 1800 to 2020
start_time = time.time()
start_year, end_year, step = 1985, 2010, 5
dfs = []
for year in range(start_year, end_year+1, step):
    
    # < IS MUCH FASTER THAN <=
    print("Running PageRank on works from < {}...".format(year), end=" ")
    query_start_time = time.time()
    query = """
    CALL algo.pageRank.stream(
    'MATCH (p:Quanta) WHERE p.year < {} RETURN id(p) as id',
    'MATCH (p1:Quanta)-[:CITES]->(p2:Quanta) RETURN id(p1) as source, id(p2) as target',
    {{graph:'cypher', iterations:20, write:false, concurrency:20}})
    YIELD node, score
    WITH 
        *,
        node.id AS id, 
        node.title AS title, 
        node.lang AS lang, 
        node.year AS year, 
        node.keywords AS keywords, 
        node.fos AS fos, 
        node.publisher AS publisher,
        score AS page_rank
    ORDER BY page_rank DESC
    LIMIT 10000000
    RETURN 
        id,
        title,
        lang, 
        year, 
        keywords, 
        fos, 
        publisher,
        page_rank;    
    """.format(year,year,year)
    
    #graph.run(query)
    #print(query)
    
    df = graph.run(query).to_data_frame()
    
print("Finished all calculations in {:.2f} minutes.".format((time.time()-start_time)/60))


Unauthorized: http://localhost:7474/db/data/

In [3]:
def normalize(df):
    mean = df.groupby('year').Score.mean()
    stDev = df.groupby('year').Score.std()
    
    scores_minus_mean = df - mean
    scaled_scores = scores_minus_mean.divide(stDev, axis=1)
    
    scaled_df = pd.DataFrame(scaled_scores)
    sorted_df = scaled_df.sort_values(['scores'], ascending=False).head(1000)
    
    return sorted_df

In [4]:
def get_top_papers(all_years_df): 
    result = all_years_df.sort_values(['scores'], ascending=False).head(1000)
    return result

In [5]:
# Get the top 1,000 scaled scores
scaled_pr = normalize(df)
results = get_top_papers(scaled_pr)

NameError: name 'df' is not defined

In [95]:
# Extra stuff 

# Make fake data
a = {'scores': [1, 2, 4, 18, 25, 13]}
df1950 = pd.DataFrame(data=a)
b = {'scores': [2, 7, 4, 68, 20, 13]}
df1951 = pd.DataFrame(data=b)
c = {'scores': [9, 8, 4, 18, 5, 83]}
df1952 = pd.DataFrame(data=c)

# Normalize each dataframe which represents pagerank scores of papers published in a given year
d1 = normalize_scores_for_year(df1950)
d2 = normalize_scores_for_year(df1951)
d3 = normalize_scores_for_year(df1952)

years = [d1, d2, d3]
#years = ['df_' + year for year in range(1900, 2000)]

# Find the top papers from all the years combined and return the top 1000
all_years_df = combine_scores(years)
result = get_top_papers(all_years_df)
    
print(result)

def normalize_scores_for_year_df(df):
    
    mean = df['scores'].mean(axis=0)
    std =  df['scores'].std(axis=0)
    #print(mean)
    #print(std)

    scores_minus_mean = df - mean
    scaled_scores = scores_minus_mean.divide(std, axis=1)

    #print(df)
    #print(scaled_scores)

    scaled_df = pd.DataFrame(scaled_scores)
    sorted_df = scaled_df.sort_values(['scores'], ascending=False).head(1000)
    
    return sorted_df

def combine_scores(dfs):
    all_years_df = pd.concat(dfs)
    return all_years_df

#d = {'paper': ['A', 'BB', 'C', 'd', 'E', 'f'], 'scores': [1, 2, 4, 18, 25, 13]}
d = {'scores': [1, 2, 4, 18, 25, 13]}
#data2 = [{'scores': scores}]#, {'a': 5, 'b': 10, 'c': 20}, {'a': 0, 'b': 8, 'c': 30}]
df = pd.DataFrame(data=d)

